In [1]:
print('ok')

ok


# Model Training


In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

### Import the CSV Data as Pandas DataFrame

In [33]:
df = pd.read_csv(r"C:\Users\LENOVO\MachineLearningProhects\StudentPerformancePrediction-End-To-End\data\raw\student.csv")

Show Top 5 Records

In [34]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [35]:
X = df.drop(columns=['math_score'],axis=1)

In [36]:
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [37]:
print("Categories in 'gender' variable:     ",end=" " )
print(df['gender'].unique())

print("Categories in 'race_ethnicity' variable:  ",end=" ")
print(df['race_ethnicity'].unique())

print("Categories in'parental level of education' variable:",end=" " )
print(df['parental_level_of_education'].unique())

print("Categories in 'lunch' variable:     ",end=" " )
print(df['lunch'].unique())

print("Categories in 'test preparation course' variable:     ",end=" " )
print(df['test_preparation_course'].unique())

Categories in 'gender' variable:      ['female' 'male']
Categories in 'race_ethnicity' variable:   ['group B' 'group C' 'group A' 'group D' 'group E']
Categories in'parental level of education' variable: ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
Categories in 'lunch' variable:      ['standard' 'free/reduced']
Categories in 'test preparation course' variable:      ['none' 'completed']


In [39]:
y = df['math_score']

In [40]:
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math_score, Length: 1000, dtype: int64

In [47]:
# Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

In [48]:
num_features

Index(['reading_score', 'writing_score'], dtype='object')

In [49]:
cat_features

Index(['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch',
       'test_preparation_course'],
      dtype='object')

In [51]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
import joblib

# Define transformers
nume_transformer = StandardScaler()
oh_transformer = OneHotEncoder(handle_unknown="ignore")

# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ("onehot", oh_transformer, cat_features),
        ("scaler", nume_transformer, num_features),        
    ]
)

# ✅ Fit preprocessor on raw data
X_transformed = preprocessor.fit_transform(X)

# ✅ Save fitted preprocessor
joblib.dump(preprocessor, "preprocessor.pkl")
print("Preprocessor saved as 'preprocessor.pkl'")

Preprocessor saved as 'preprocessor.pkl'


In [52]:
X_transformed

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]], shape=(1000, 19))

In [55]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_transformed,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

### Create an Evaluate Function to give all metrics after model Training

In [56]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# -----------------------
# Utility Functions
# -----------------------

def evaluate_model(true, predicted):
    """Return MAE, RMSE, and R2 score"""
    mae = mean_absolute_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2 = r2_score(true, predicted)
    return {"MAE": mae, "RMSE": rmse, "R2": r2}

def train_and_evaluate(model, X_train, X_test, y_train, y_test):
    """Train model, predict, and evaluate"""
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    return {
        "Train": evaluate_model(y_train, y_train_pred),
        "Test": evaluate_model(y_test, y_test_pred),
    }

def hyperparameter_tuning(model, param_grid, X_train, y_train, search_type="grid", cv=3, n_iter=20):
    """Perform hyperparameter tuning using GridSearchCV or RandomizedSearchCV"""
    if search_type == "grid":
        search = GridSearchCV(model, param_grid, cv=cv, scoring="r2", n_jobs=-1, verbose=0)
    else:
        search = RandomizedSearchCV(model, param_grid, cv=cv, scoring="r2", n_iter=n_iter, n_jobs=-1, random_state=42, verbose=0)

    search.fit(X_train, y_train)
    return search.best_estimator_, search.best_params_

# -----------------------
# Models and Parameters
# -----------------------

models_and_params = {
    "Linear Regression": (LinearRegression(), {}),
    "Lasso": (Lasso(), {"alpha": [0.01, 0.1, 1, 10]}),
    "Ridge": (Ridge(), {"alpha": [0.01, 0.1, 1, 10]}),
    "KNN": (KNeighborsRegressor(), {"n_neighbors": [3, 5, 7, 9]}),
    "Decision Tree": (DecisionTreeRegressor(random_state=42), {"max_depth": [3, 5, 10, None]}),
    "Random Forest": (RandomForestRegressor(random_state=42), {"n_estimators": [100, 200], "max_depth": [None, 10, 20]}),
    "XGBoost": (XGBRegressor(random_state=42, eval_metric="rmse"), {"n_estimators": [100, 200], "max_depth": [3, 5, 7]}),
    "CatBoost": (CatBoostRegressor(random_state=42, verbose=0), {"depth": [4, 6, 8], "learning_rate": [0.01, 0.05, 0.1]}),
    "AdaBoost": (AdaBoostRegressor(random_state=42), {"n_estimators": [50, 100, 200]})
}

# -----------------------
# Main Pipeline
# -----------------------

def run_pipeline(X, y, test_size=0.2, random_state=42, tuning=True):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    results = []

    for name, (model, params) in models_and_params.items():
        print(f"🔹 Training {name}...")

        if tuning and params:  # If tuning is enabled and params exist
            best_model, best_params = hyperparameter_tuning(model, params, X_train, y_train, search_type="random")
            print(f"Best Params for {name}: {best_params}")
            model = best_model

        scores = train_and_evaluate(model, X_train, X_test, y_train, y_test)

        results.append({
            "Model": name,
            "Train_R2": scores["Train"]["R2"],
            "Test_R2": scores["Test"]["R2"],
            "Test_RMSE": scores["Test"]["RMSE"],
            "Test_MAE": scores["Test"]["MAE"],
        })

    results_df = pd.DataFrame(results).sort_values(by="Test_R2", ascending=False).reset_index(drop=True)
    return results_df

# -----------------------
# Example Usage
# -----------------------
# results_df = run_pipeline(X, y, tuning=True)
# print(results_df)


In [27]:
results_df = run_pipeline(X, y, tuning=True)
print(results_df)

🔹 Training Linear Regression...
🔹 Training Lasso...


c:\Users\LENOVO\miniconda3\envs\myenv\lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 4 is smaller than n_iter=20. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params for Lasso: {'alpha': 0.01}
🔹 Training Ridge...
Best Params for Ridge: {'alpha': 1}
🔹 Training KNN...


c:\Users\LENOVO\miniconda3\envs\myenv\lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 4 is smaller than n_iter=20. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\LENOVO\miniconda3\envs\myenv\lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 4 is smaller than n_iter=20. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params for KNN: {'n_neighbors': 9}
🔹 Training Decision Tree...
Best Params for Decision Tree: {'max_depth': 5}
🔹 Training Random Forest...


...

Best Params for XGBoost: {'n_estimators': 100, 'max_depth': 3}
🔹 Training CatBoost...


c:\Users\LENOVO\miniconda3\envs\myenv\lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 9 is smaller than n_iter=20. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params for CatBoost: {'learning_rate': 0.01, 'depth': 4}
🔹 Training AdaBoost...


c:\Users\LENOVO\miniconda3\envs\myenv\lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 3 is smaller than n_iter=20. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params for AdaBoost: {'n_estimators': 200}
               Model  Train_R2   Test_R2  Test_RMSE  Test_MAE
0              Lasso  0.874274  0.880888   5.383737  4.204596
1              Ridge  0.874304  0.880593   5.390387  4.211101
2  Linear Regression  0.874317  0.880433   5.393994  4.214763
3           CatBoost  0.888959  0.871438   5.593224  4.280585
4            XGBoost  0.931863  0.864628   5.739440  4.384700
5      Random Forest  0.965851  0.855180   5.936358  4.592756
6           AdaBoost  0.854845  0.853037   5.980117  4.628317
7      Decision Tree  0.855011  0.824230   6.540001  4.931523
8                KNN  0.840059  0.790877   7.133563  5.569444


In [57]:
print(results_df)

               Model  Train_R2   Test_R2  Test_RMSE  Test_MAE
0              Lasso  0.874274  0.880888   5.383737  4.204596
1              Ridge  0.874304  0.880593   5.390387  4.211101
2  Linear Regression  0.874317  0.880433   5.393994  4.214763
3           CatBoost  0.888959  0.871438   5.593224  4.280585
4            XGBoost  0.931863  0.864628   5.739440  4.384700
5      Random Forest  0.965851  0.855180   5.936358  4.592756
6           AdaBoost  0.854845  0.853037   5.980117  4.628317
7      Decision Tree  0.855011  0.824230   6.540001  4.931523
8                KNN  0.840059  0.790877   7.133563  5.569444


In [58]:
# 1️⃣ Import joblib
import joblib
from sklearn.linear_model import Lasso

# 2️⃣ Train your Lasso model (if not already trained)
lasso_model = Lasso(alpha=0.1, random_state=42)  # you can set alpha from tuning
lasso_model.fit(X_train, y_train)

# 3️⃣ Save the trained model to a file
joblib.dump(lasso_model, "lasso_model.pkl")
print("Model saved as 'lasso_model.pkl'")

# 4️⃣ Load the model later for prediction
loaded_model = joblib.load("lasso_model.pkl")
y_pred = loaded_model.predict(X_test)


Model saved as 'lasso_model.pkl'


In [61]:
user_input = pd.DataFrame([{
    "gender": "female",
    "race_ethnicity": "group B",
    "parental_level_of_education": "bachelor's degree",
    "lunch": "standard",
    "test_preparation_course": "none",
    "reading_score": 72,
    "writing_score": 74
}])

In [62]:
# Load both
preprocessor = joblib.load("preprocessor.pkl")
model = joblib.load("lasso_model.pkl")

# Transform user input
X_user = preprocessor.transform(user_input)

# Predict
predicted_score = model.predict(X_user)
print("🎯 Predicted Math Score:", predicted_score[0])


🎯 Predicted Math Score: 66.33453886317913
